In [ ]:
import textwrap 
import requests
import tkinter as tk
from tkinter import ttk
from prettytable import PrettyTable

class CPICGUI:
    def __init__(self, master):
        self.master = master
        master.title("CPIC PGx API GUI")

        self.gene_label = ttk.Label(master, text="Gene Symbol:")
        self.gene_label.grid(row=0, column=0, padx=5, pady=5, sticky=tk.W)

        self.gene_entry = ttk.Entry(master)
        self.gene_entry.grid(row=0, column=1, padx=5, pady=5)

        self.lookup_button = ttk.Button(master, text="Get Lookupkeys", command=self.fetch_lookupkeys)
        self.lookup_button.grid(row=0, column=2, padx=5, pady=5)

        self.lookup_label = ttk.Label(master, text="Lookupkeys:")
        self.lookup_label.grid(row=1, column=0, padx=5, pady=5, sticky=tk.W)

        self.lookup_listbox = tk.Listbox(master,height=5, width=40)
        self.lookup_listbox.grid(row=2, column=0, columnspan=3, padx=5, pady=5)

        self.sel_lookup_button = ttk.Button(master, text="Select Lookupkey", command=self.select_lookupkey)
        self.sel_lookup_button.grid(row=2, column=2, padx=5, pady=5)
        
        self.drug_label = ttk.Label(master, text="Drug Name:")
        self.drug_label.grid(row=3, column=0, padx=5, pady=5, sticky=tk.W)

        self.drug_entry = ttk.Entry(master)
        self.drug_entry.grid(row=3, column=1, padx=5, pady=5)

        self.lookupkey_label = ttk.Label(master, text="Lookupkey:")
        self.lookupkey_label.grid(row=4, column=0, padx=5, pady=5, sticky=tk.W)

        self.lookupkey_entry = ttk.Entry(master,width=40)
        self.lookupkey_entry.grid(row=4, column=0, columnspan=3,padx=5, pady=5 )

        self.recommend_button = ttk.Button(master, text="Get Recommendations", command=self.get_recommendations)
        self.recommend_button.grid(row=4, column=2, padx=5, pady=5)

        self.recommend_label = ttk.Label(master, text="Recommendations:")
        self.recommend_label.grid(row=5, column=0, padx=5, pady=5, sticky=tk.W)
        
        self.recommend_table = ttk.Treeview(master, columns=('Drug', 'Guideline', 'Recommendation'), show='headings', height=4)
        self.scrollbar_x = ttk.Scrollbar(master, orient='horizontal', command=self.recommend_table.xview)
        self.scrollbar_x.grid(row=7, column=0, columnspan=3, sticky='ew', padx=5, pady=5)
        self.recommend_table.configure(xscrollcommand=self.scrollbar_x.set)

        
        self.recommend_table.column('Guideline', width=400, anchor="nw", stretch=True, minwidth=200)
        self.recommend_table.column('Recommendation', anchor="nw",width=400,stretch=True, minwidth=200)
        self.recommend_table.grid(row=6, column=0, columnspan=3, padx=5, pady=5)

        self.recommend_table.heading('Drug', text='Drug')
        self.recommend_table.heading('Guideline', text='Guideline')
        self.recommend_table.heading('Recommendation', text='Recommendation')
        self.recommend_scrollbar = ttk.Scrollbar(master, orient="vertical", command=self.recommend_table.yview)
        self.recommend_scrollbar.grid(row=6, column=3, sticky="ns")
        self.recommend_table.configure(yscrollcommand=self.recommend_scrollbar.set)
        
        self.clear_button = tk.Button(master, text="Clear Recommendations", command=self.clear_recommendations)
        self.clear_button.grid(row=5, column=2, sticky="ns")
        # Set column width
        #tree.column('A', width=200)

    def clear_recommendations(self):
        for item in self.recommend_table.get_children():
            self.recommend_table.delete(item)

      
    def select_lookupkey(self):
        self.lookupkey_entry.delete(0, tk.END)
        self.lookupkey_entry.insert(0,self.lookup_listbox.get(self.lookup_listbox.curselection()))
    def fetch_lookupkeys(self):
        self.lookup_listbox.delete(0, tk.END)
        gene_symbol = self.gene_entry.get()
        lookupkeys = self.get_lookupkeys(gene_symbol)
        if lookupkeys:
            for lookupkey in lookupkeys:
                gene = list(lookupkey.keys())[0]
                phenotype = list(lookupkey.values())[0]
                self.lookup_listbox.insert(tk.END, f"{gene}: {phenotype}")
        else:
            self.lookup_listbox.insert(tk.END, "No lookupkeys found.")

    def get_lookupkeys(self,gene_symbol):
        url = f"https://api.cpicpgx.org/v1/diplotype?genesymbol=eq.{gene_symbol}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            unique_lookupkeys = []
            for item in data:
                lookupkey = item['lookupkey']
                if lookupkey not in unique_lookupkeys:
                    unique_lookupkeys.append(lookupkey)
            return unique_lookupkeys
        return None
    def get_recommendations(self):
        drugname= self.drug_entry.get()
        url = f"https://api.cpicpgx.org/v1/drug?name=eq.{drugname}"
        response = requests.get(url)
        if response.ok:
            data = response.json()
            drugid = data[0]['drugid']
        else:
            self.recommend_table.insert('', tk.END, values=("No recommendations found.", "", ""))
        lookupkey = self.lookupkey_entry.get()
        lookupkey = '{"'+lookupkey.split(":")[0]+'": "'+lookupkey.split(":")[1][1:]+'"}'
        url = f"https://api.cpicpgx.org/v1/recommendation?select=drug(name), guideline(name),* &drugid=eq.{drugid}&lookupkey=cs.{lookupkey}"
        response = requests.get(url)
        print(url)
        if response.ok:
            print("Success")
            data = response.json()
            print(data)
            recommendations = [{ 'Drug': item['drug']['name'], 'Guideline': item['guideline']['name'], 'Recommendation': item['drugrecommendation'] } for item in data]
            for rec in recommendations:
                print(rec)     
                #                    
                #values=(rec['Drug'], rec['Guideline'], rec['Recommendation'])
                #self.recommend_table.insert('', tk.END, values=list(map(wrap, values)))
                self.recommend_table.insert('', tk.END, values=(rec['Drug'], rec['Guideline'], rec['Recommendation']),tags=("wrapped",))
        else:
            self.recommend_table.insert('', tk.END, values=("No recommendations found.", "", ""))
        self.recommend_table.see(self.recommend_table.get_children()[-1])
        return None

def wrap(string, lenght=50):
    return '\n'.join(textwrap.wrap(string, lenght))

def print_table(headers, rows):
    table = PrettyTable(headers)
    for row in rows:
        table.add_row(row)
    print(table)

def get_recommendation(drug_name, lookupkey):
    url = f"https://api.cpicpgx.org/v1/recommendation?select=drugid(drug(name)), guideline(name),* &drug(name)=eq.{drug_name}&lookupkey=cs.{lookupkey}"
    response = requests.get(url)
    if response.ok:
        data = response.json()
        recommendations = [{ 'Drug': item['drug']['name'], 'Guideline': item['guideline']['name'], 'Recommendation': item['drugrecommendation'] } for item in data]
        return recommendations
    else:
        print(f"Error: {response.status_code}\n{response.content}")
        return None

if __name__ == '__main__':
    root = tk.Tk()
    gui = CPICGUI(root)
   
    root.mainloop()
